In [1]:
import pandas as pd
import numpy as np

import copy

from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_parquet('../../data/final_data/fbref/player_match_reports.parquet')
df['Date'] = pd.to_datetime(df['Date'], format = "%Y-%m-%d")
df

,Season,League,Team,Date,Player,#,Nation,Pos,Age,Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Aerial_Duels_Won%,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2020-2021,EPL,Fulham,2020-09-12,Aboubakar Kamara,47,mr MTN,FW,25-189,62.0,0,0,0,0,1,0,0,0,26.0,13.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,16.0,87.5,0.0,20.0,2.0,0.0,2.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,13.0,0.0,0.0,1.0,5.0,7.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,3,1,0,0,0,0.0,0.0,0,4.0,2.0,1.0,66.7,14.0,16.0,87.5,189.0,12.0,4.0,5.0,80.0,7.0,8.0,87.5,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,12.0,4.0,0.0,0.0,5.0,0.0,0,0.0,0.0,0.0,0.0,11.0,4.0,1.0,4.0,10.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,1.0,26.0,0.0,1.0,13.0,14.0,1.0,22.0,0.0,0.0,0.0,45.0,23.0,0.0,0.0,3.0,4.0,26.0,18.0,69.2,2.0
1,2020-2021,EPL,Fulham,2020-09-12,Aleksandar Mitrović,9,rs SRB,FW,25-362,28.0,0,0,0,0,1,1,0,0,7.0,6.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,60.0,0.0,6.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,6.0,0.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,2,0,1,0,0.0,0.0,0,0.0,0.0,0.0,NaN,3.0,5.0,60.0,44.0,2.0,2.0,3.0,66.7,1.0,1.0,100.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,2.0,1.0,1,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,3.0,0.0,0.0,1.0,3.0,0.0,1.0,1.0,0.0,7.0,0.0,0.0,3.0,4.0,0.0,7.0,100.0,1.0,0.0,16.0,3.0,0.0,0.0,1.0,0.0,12.0,7.0,58.3,1.0
2,2020-2021,EPL,Fulham,2020-09-12,Ivan Cavaleiro,17,pt POR,LW,26-330,90.0,0,0,0,0,0,0,0,0,41.0,14.0,0.0,0,1.0,0.0,0.0,0.1,3.0,0.0,28.0,35.0,80.0,2.0,30.0,4.0,3.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,14.0,1.0,7.1,2.0,5.0,7.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0.0,0,1,4,0,2,0,0.0,0.0,0,6.0,1.0,0.0,100.0,28.0,35.0,80.0,485.0,84.0,14.0,15.0,93.3,11.0,13.0,84.6,3.0,6.0,50.0,0,0.1,2.0,1.0,1.0,1.0,2.0,35.0,34.0,1.0,1.0,0.0,7.0,2.0,2,0.0,0.0,0.0,0.0,27.0,2.0,6.0,4.0,29.0,2.0,0.0,0.0,28.0,0.0,0.0,0.0,1.0,41.0,4.0,11.0,23.0,11.0,0.0,41.0,75.0,3.0,0.0,189.0,76.0,2.0,0.0,2.0,1.0,41.0,34.0,82.9,1.0
3,2020-2021,EPL,Fulham,2020-09-12,Neeskens Kebano,7,cd COD,RW,28-186,62.0,0,0,0,0,1,0,0,0,30.0,9.0,1.0,0,2.0,0.1,0.1,0.0,1.0,0.0,19.0,23.0,82.6,1.0,21.0,3.0,2.0,3.0,1.0,1,1.0,0.0,0.0,1.0,2.0,50.0,1.0,9.0,3.0,33.3,1.0,6.0,2.0,2.0,0.0,0.0,2.0,0,1.0,1.0,0.0,0,0,2,1,1,1,0.0,0.0,0,4.0,0.0,0.0,NaN,19.0,23.0,82.6,215.0,39.0,15.0,15.0,100.0,3.0,6.0,50.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,23.0,22.0,1.0,1.0,0.0,6.0,0.0,1,0.0,0.0,0.0,0.0,17.0,4.0,2.0,8.0,15.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,30.0

In [4]:
df[df['Date'] == '2019-09-28']['Team'].unique()

array(['Sheffield Utd', 'Liverpool', 'Crystal Palace', 'Norwich City',
       'Aston Villa', 'Burnley', 'Wolves', 'Watford', 'Tottenham',
       'Southampton', 'Chelsea', 'Brighton', 'Bournemouth', 'West Ham',
       'Everton', 'Manchester City', 'Lyon', 'Nantes', 'Bordeaux',
       'Paris S-G', 'Metz', 'Toulouse', 'Reims', 'Dijon', 'Monaco',
       'Brest', 'Angers', 'Amiens', 'Nice', 'Lille', 'Paderborn 07',
       'Bayern Munich', 'RB Leipzig', 'Schalke 04', 'Augsburg',
       'Leverkusen', 'Mainz 05', 'Wolfsburg', 'Hoffenheim', "M'Gladbach",
       'Dortmund', 'Werder Bremen', 'Juventus', 'SPAL', 'Sampdoria',
       'Inter', 'Sassuolo', 'Atalanta', 'Athletic Club', 'Valencia',
       'Getafe', 'Barcelona', 'Granada', 'Leganés', 'Atlético Madrid',
       'Real Madrid'], dtype=object)

In [5]:
map_dict = {'DM': 'M',
 'CB': 'D',
 'LB': 'D',
 'RM': 'M',
 'DF': 'D',
 'RB': 'D',
 'AM': 'M',
 'MF': 'M',
 'CM': 'M',
 'LW': 'A',
 'FW': 'A',
 'RW': 'A',
 'WB': 'D',
 'GK': 'G',
 'LM': 'M'}
new_pos = []
for i, row in df[['Pos']].iterrows():
    pos = row['Pos']
    counts = {
        'D': 0,
        'M': 0,
        'A': 0, 
        'G': 0
    }
    for p in pos.split(','):
        counts[map_dict[p]] += 1
    max_val = 0
    best_key = 'M'
    for key in counts:
        if counts[key] > max_val:
            max_val = counts[key]
            best_key = key
    new_pos.append(best_key)
    
#     print(pos, best_key)
df['PosNew'] = new_pos

In [6]:
columns_to_be_dropped = [
    'Nation',
    '#',
    'Performance_Gls',
    'Performance_Ast',
    'Performance_PK', 
    'Performance_PKatt',
] + [col for col in df.columns if '%' in col ]

df = df.drop(columns_to_be_dropped, 1)

In [7]:
cols_to_be_normalized = list(set(df.columns) - set(['Season',
 'League',
 'Team',
 'Date',
 'Player',
 'Pos',
 'PosNew',
 'Age']))
df[cols_to_be_normalized] = df[cols_to_be_normalized].div(df['Min'], axis=0)

In [8]:
df[(df['Season'] == '2017-2018') & (df.isna().any(axis=1))]

,Season,League,Team,Date,Player,Pos,Age,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,PosNew
32057,2017-2018,EPL,Tottenham,2017-10-14,Georges-Kévin N'Koudou,AM,22-243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
34714,2017-2018,EPL,West Brom,2017-12-13,Sam Field,MF,19-219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
40429,2017-2018,EPL,Manchester City,2018-05-13,Ederson,GK,24-269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G
69680,2017-2018,Ligue1,Monaco,2017-08-18,Youri Tielemans,FW,20-103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
69763,2017-2018,Ligue1,Nantes,2017-08-19,Mariusz Stępiński,FW,22-099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [9]:
df[(df['Date'] == '2019-09-28') & (df['Team'] == 'Metz')]

,Season,League,Team,Date,Player,Pos,Age,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,PosNew
52832,2019-2020,Ligue1,Metz,2019-09-28,Habib Diallo,FW,24-102,1.0,0.090909,0.090909,0.000000,0.0,0.363636,0.090909,0.000000,0.000000,0.000000,0.009091,0.009091,0.009091,0.181818,0.000000,0.181818,0.181818,0.000000,0.363636,NaN,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.181818,0.181818,3.545455,0.000000,0.090909,0.090909,0.090909,0.090909,0.000000,0.000000,0.000000,0.009091,0.090909,0.000000,0.000000,0.000000,0.000000,0.181818,0.090909,0.090909,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.181818,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.181818,0.0,0.000000,0.000000,0.000000,0.363636,0.000000,0.000000,0.090909,0.272727,0.181818,0.272727,0.090909,0.0,3.090909,1.909091,NaN,NaN,0.000000,0.090909,0.545455,0.272727,0.090909,A
52833,2019-2020,Ligue1,Metz,2019-09-28,Ibrahima Niane,FW,20-201,1.0,0.025316,0.000000,0.000000,0.0,0.518987,0.253165,0.025316,0.000000,0.012658,0.011392,0.001266,0.000000,0.012658,0.000000,0.227848,0.341772,0.000000,0.303797,NaN,0.012658,0.012658,0.025316,0.012658,0.025316,0.000000,0.000000,0.000000,0.000000,0.000000,0.253165,0.050633,0.037975,0.113924,0.101266,0.012658,0.000000,0.0,0.012658,0.000000,0.025316,0.012658,0.0,0.0,0.025316,0.012658,0.0,0.000000,0.012658,0.0,0.0,0.0,0.012658,0.050633,0.012658,0.227848,0.341772,3.810127,0.354430,0.088608,0.151899,0.126582,0.177215,0.000000,0.000000,0.000000,0.000000,0.012658,0.012658,0.000000,0.000000,0.000000,0.341772,0.316456,0.025316,0.000000,0.0,0.113924,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.215190,0.075949,0.050633,0.050633,0.189873,0.050633,0.000000,0.000000,0.227848,0.0,0.000000,0.025316,0.000000,0.518987,0.012658,0.063291,0.202532,0.253165,0.063291,0.481013,0.012658,0.0,1.227848,0.455696,NaN,NaN,0.075949,0.000000,0.759494,0.405063,0.139241,A
52834,2019-2020,Ligue1,Metz,2019-09-28,Opa Nguette,LW,25-082,1.0,0.022222,0.022222,0.000000,0.0,0.511111,0.200000,0.044444,0.000000,0.000000,0.006667,0.006667,0.000000,0.044444,0.000000,0.355556,0.377778,0.000000,0.488889,NaN,0.022222,0.022222,0.044444,0.044444,0.022222,0.022222,0.000000,0.022222,0.022222,0.000000,0.200000,0.088889,0.066667,0.088889,0.044444,0.000000,0.000000,0.0,0.00

In [10]:
df = df.dropna()

In [11]:
df[df['Date'] == '2019-09-28']['Team'].unique()

array(['Sheffield Utd', 'Liverpool', 'Crystal Palace', 'Norwich City',
       'Aston Villa', 'Burnley', 'Wolves', 'Watford', 'Tottenham',
       'Southampton', 'Chelsea', 'Brighton', 'Bournemouth', 'West Ham',
       'Everton', 'Manchester City', 'Lyon', 'Nantes', 'Bordeaux',
       'Paris S-G', 'Reims', 'Dijon', 'Monaco', 'Brest', 'Angers',
       'Amiens', 'Nice', 'Lille', 'Paderborn 07', 'Bayern Munich',
       'RB Leipzig', 'Schalke 04', 'Augsburg', 'Leverkusen', 'Mainz 05',
       'Wolfsburg', 'Hoffenheim', "M'Gladbach", 'Dortmund',
       'Werder Bremen', 'Juventus', 'SPAL', 'Sampdoria', 'Inter',
       'Sassuolo', 'Atalanta', 'Athletic Club', 'Valencia', 'Getafe',
       'Barcelona', 'Granada', 'Leganés', 'Atlético Madrid',
       'Real Madrid'], dtype=object)

In [12]:
matches_df = pd.read_csv('../../data/final_data/fbref/matches.csv', parse_dates=['Date'])
matches_df = matches_df.rename(columns={
    'HomeScore': 'Goal_Home', 
    'AwayScore': 'Goal_Away', 
    'HomeTeam': 'Team_Home',
    'AwayTeam': 'Team_Away',
})
matches_df['Win_Home'] = 1 * ((matches_df['Goal_Home'] - matches_df['Goal_Away']) > 0)
matches_df['Win_Away'] = 1 * ((- matches_df['Goal_Home'] + matches_df['Goal_Away']) > 0)
matches_df['game_id'] = matches_df.index
matches_df.head()

,League,Season,Wk,Day,Date,Time,Team_Home,xG_Home,xG_Away,Team_Away,Attendance,Venue,Referee,Match Report,Goal_Home,Goal_Away,Round,Win_Home,Win_Away,game_id
0,EPL,2020-2021,1.0,Sat,2020-09-12,12:30,Fulham,0.2,1.8,Arsenal,NaN,Craven Cottage,Chris Kavanagh,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,3.0,NaN,0,1,0
1,EPL,2020-2021,1.0,Sat,2020-09-12,15:00,Crystal Palace,0.7,0.8,Southampton,NaN,Selhurst Park,Jonathan Moss,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.0,NaN,1,0,1
2,EPL,2020-2021,1.0,Sat,2020-09-12,17:30,Liverpool,3.3,0.6,Leeds United,NaN,Anfield,Michael Oliver,https://fbref.com/en/matches/21b58926/Liverpoo...,4.0,3.0,NaN,1,0,2
3,EPL,2020-2021,1.0,Sat,2020-09-12,20:00,West Ham,1.1,1.5,Newcastle Utd,NaN,London Stadium,Stuart Attwell,https://fbref.com/en/matches/78495ced/West-Ham...,0.0,2.0,NaN,0,1,3
4,EPL,2020-2021,1.0,Sun,2020-09-13,14:00,West Brom,0.5,2.2,Leicester City,NaN,The Hawthorns,Anthony Taylor,https://fbref.com/en/matches/7dd01ca9/West-Bro...,0.0,3.0,NaN,0,1,4


In [13]:
len(matches_df)

7314

In [14]:
matches_df_long = pd.wide_to_long(matches_df, ["Goal", "xG", 'Team', 'Win'], i="game_id", j="isHome", sep='_', suffix='(!?Home|Away)')
matches_df_long = matches_df_long.drop(['Attendance', 'Round'], 1)
matches_df_long = matches_df_long.reset_index()
matches_df_long.head()


,game_id,isHome,Time,Season,Date,Day,Venue,League,Wk,Referee,Match Report,Goal,xG,Team,Win
0,0,Home,12:30,2020-2021,2020-09-12,Sat,Craven Cottage,EPL,1.0,Chris Kavanagh,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,0.2,Fulham,0
1,1,Home,15:00,2020-2021,2020-09-12,Sat,Selhurst Park,EPL,1.0,Jonathan Moss,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.7,Crystal Palace,1
2,2,Home,17:30,2020-2021,2020-09-12,Sat,Anfield,EPL,1.0,Michael Oliver,https://fbref.com/en/matches/21b58926/Liverpoo...,4.0,3.3,Liverpool,1
3,3,Home,20:00,2020-2021,2020-09-12,Sat,London Stadium,EPL,1.0,Stuart Attwell,https://fbref.com/en/matches/78495ced/West-Ham...,0.0,1.1,West Ham,0
4,4,Home,14:00,2020-2021,2020-09-13,Sun,The Hawthorns,EPL,1.0,Anthony Taylor,https://fbref.com/en/matches/7dd01ca9/West-Bro...,0.0,0.5,West Brom,0


In [15]:
len(matches_df_long)

14628

In [16]:
df_grouped = df.groupby(['League', 'Season', 'Team', 'Date', 'PosNew']).sum().reset_index()
team_stats_df = pd.merge(df_grouped, matches_df_long[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']], how='left', on=['League', 'Season', 'Team', 'Date'])
team_stats_df.head(5)


,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
0,Bundesliga,2017-2018,Augsburg,2017-08-19,A,4.0,0.057431,0.011111,0.000000,0.0,2.399867,0.728161,0.011111,0.011111,0.024098,0.003521,0.003521,0.008803,0.182551,0.000000,1.021134,1.776313,0.092641,1.810523,0.277944,0.124098,0.269564,0.011111,0.011111,0.011111,0.000000,0.000000,0.000000,0.033333,0.033333,0.728161,0.137085,0.147342,0.364080,0.216739,0.024098,0.000000,0.0,0.024098,0.011111,0.022222,0.035209,0.0,0.0,0.066667,0.055556,0.000000,0.376235,0.011111,0.0,0.0,0.0,0.464247,0.148196,0.137085,1.021134,1.776313,15.497336,3.913198,0.591242,0.757909,0.361350,0.664413,0.046320,0.261350,0.0,0.008803,0.121368,0.061183,0.143590,0.099145,0.092641,1.776313,1.591886,0.184427,0.102897,0.0,0.253824,0.011111,0.376235,0.025974,0.012987,0.012987,0.0,0.869186,0.562371,0.344755,0.522555,1.061117,0.048196,0.033333,0.022222,1.021134,0.000000,0.011111,0.000000,0.081530,2.399867,0.000000,0.138961,0.822844,1.673438,0.265956,2.215440,0.135209,0.024098,8.346831,4.807604,0.099145,0.022222,0.200167,0.046320,2.337496,1.623343,0.709713,3970,0.0,0,Away
1,Bundesliga,2017-2018,Augsburg,2017-08-19,D,4.0,0.022222,0.000000,0.022222,0.0,3.266667,0.455556,0.088889,0.033333,0.066667,0.000000,0.000000,0.003333,0.111111,0.000000,2.111111,2.755556,0.322222,1.833333,0.255556,0.033333,0.033333,0.088889,0.066667,0.066667,0.011111,0.011111,0.033333,0.066667,0.033333,0.455556,0.200000,0.233333,0.144444,0.077778,0.066667,0.022222,0.0,0.044444,0.033333,0.122222,0.233333,0.0,0.0,0.088889,0.033333,0.011111,0.133333,0.066667,0.0,0.0,0.0,0.444444,0.088889,0.122222,2.111111,2.755556,52.500000,20.277778,0.555556,0.688889,0.900000,1.022222,0.644444,0.966667,0.0,0.003333,0.077778,0.255556,0.100000,0.055556,0.322222,2.755556,2.344444,0.411111,0.077778,0.0,0.222222,0.177778,0.133333,0.055556,0.044444,0.000000,0.0,1.355556,0.655556,0.744444,1.088889,1.266667,0.100000,0.277778,0.000000,2.111111,0.000000,0.055556,0.044444,0.100000,3.266667,0.077778,1.211111,1.577778,0.755556,0.088889,2.866667,0.033333,0.000000,10.866667,6.311111,0.088889,0.011111,0.033333,0.055556,2.100000,1.966667,0.111111,3970,0.0,0,Away
2,Bundesliga,2017-2018,Augsburg,2017-08-19,G,1.0,0.011111,0.000000,0.000000,0.0,0.488889,0.000000,0.000000,0.000000,0.000000,0.001111,0.001111,0.000000,0.000000,0.000000,0.300000,0.466667,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [17]:
len(team_stats_df.groupby(['League', 'Season', 'Team', 'Date']).mean())

13958

In [18]:
home_stats_df = team_stats_df[team_stats_df['isHome'] == 'Home']
home_stats_df = home_stats_df.sort_values(['game_id', 'PosNew'])
away_stats_df = team_stats_df[team_stats_df['isHome'] == 'Away']
away_stats_df = away_stats_df.sort_values(['game_id', 'PosNew'])

In [19]:
home_stats_df.head(5)

,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
19552,EPL,2020-2021,Fulham,2020-09-12,A,5.0,0.067972,0.035714,0.000000,0.0,2.430210,1.010394,0.051843,0.000000,0.114798,0.001613,0.001613,0.001111,0.085177,0.0,1.521941,1.803635,0.074066,1.851767,0.196518,0.137020,0.196518,0.051843,0.016129,0.016129,0.000000,0.035714,0.051843,0.119816,0.067972,1.010394,0.095212,0.233052,0.375832,0.401510,0.114798,0.035714,0.000000,0.079083,0.000000,0.051843,0.027240,0.0,0.0,0.095212,0.164260,0.016129,0.074066,0.016129,0.0,0.0,0.0,0.267127,0.043369,0.051843,1.521941,1.803635,23.119304,4.684485,0.712007,0.774962,0.640655,0.763108,0.069048,0.154224,0.0,0.001111,0.057937,0.082540,0.011111,0.011111,0.074066,1.803635,1.711879,0.091756,0.027240,0.0,0.398054,0.057937,0.074066,0.000000,0.000000,0.000000,0.0,1.430184,0.222683,0.150768,0.309421,1.404019,0.022222,0.000000,0.035714,1.521941,0.000000,0.035714,0.071429,0.027240,2.430210,0.115873,0.329595,1.191039,1.018536,0.032258,2.365694,0.137020,0.0,9.987097,4.417025,0.038351,0.000000,0.138582,0.147056,2.437122,1.880082,0.163185,0,0.0,0,Home
19553,EPL,2020-2021,Fulham,2020-09-12,D,4.0,0.011111,0.011111,0.011111,0.0,3.077778,0.433333,0.077778,0.011111,0.122222,0.000000,0.000000,0.000000,0.011111,0.0,2.344444,2.788889,0.100000,1.766667,0.177778,0.000000,0.000000,0.077778,0.044444,0.033333,0.044444,0.000000,0.044444,0.066667,0.022222,0.433333,0.111111,0.255556,0.144444,0.033333,0.122222,0.033333,0.011111,0.088889,0.011111,0.088889,0.100000,0.0,0.0,0.055556,0.011111,0.011111,0.033333,0.044444,0.0,0.0,0.0,0.300000,0.022222,0.022222,2.344444,2.788889,45.166667,16.144444,0.966667,1.044444,1.033333,1.155556,0.322222,0.477778,0.0,0.000000,0.000000,0.133333,0.000000,0.000000,0.100000,2.788889,2.488889,0.300000,0.066667,0.0,0.244444,0.022222,0.033333,0.011111,0.011111,0.000000,0.0,2.066667,0.377778,0.344444,1.155556,1.311111,0.088889,0.222222,0.000000,2.344444,0.022222,0.066667,0.011111,0.088889,3.077778,0.377778,1.322222,1.455556,0.466667,0.011111,2.800000,0.000000,0.0,8.966667,4.555556,0.066667,0.000000,0.011111,0.000000,2.233333,2.188889,0.022222,0,0.0,0,Home
19554,EPL,2020-2021,Fulham,2020-09-12,G,1.0,0.000000,0.000000,0.000000,0.0,0.311111,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211111,0.288889,0.000000,0.144444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.011111,0.022222,0.000000,0.022222,0.000000,0.000000,0.000000,

In [20]:
away_stats_df.head(5)

,League,Season,Team,Date,PosNew,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog,game_id,Goal,Win,isHome
18641,EPL,2020-2021,Arsenal,2020-09-12,A,5.0,0.083256,0.047183,0.011111,0.0,2.365504,1.438114,0.024961,0.000000,0.089922,0.017096,0.017096,0.014103,0.201034,0.040000,1.541163,1.830413,0.106667,1.691447,0.213333,0.045478,0.134367,0.024961,0.024961,0.013333,0.011628,0.000000,0.000000,0.033850,0.033850,1.438114,0.421473,0.115401,1.006951,0.315762,0.089922,0.000000,0.0,0.089922,0.000000,0.024961,0.013333,0.000000,0.0,0.047183,0.095401,0.022739,0.113333,0.024961,0.0,0.0,0.0,0.187700,0.011111,0.000000,1.541163,1.830413,24.151163,5.367390,0.806434,0.943023,0.515401,0.559845,0.196072,0.243773,0.026667,0.014103,0.129406,0.064444,0.091111,0.024444,0.106667,1.830413,1.645452,0.184961,0.026667,0.000000,0.277623,0.038295,0.113333,0.146667,0.066667,0.013333,0.0,1.453463,0.187545,0.189406,0.633850,1.138269,0.011111,0.000000,0.036072,1.541163,0.011111,0.023256,0.011628,0.084961,2.365504,0.013333,0.399406,0.912196,1.152196,0.365323,2.167209,0.056589,0.000000,7.332894,3.707907,0.062222,0.035556,0.102739,0.059328,2.791111,1.851292,0.418501,0,3.0,1,Away
18642,EPL,2020-2021,Arsenal,2020-09-12,D,5.0,0.011111,0.011111,0.011111,0.0,4.888889,0.611111,0.066667,0.022222,0.088889,0.001111,0.001111,0.002222,0.044444,0.011111,4.055556,4.522222,0.244444,3.477778,0.288889,0.055556,0.111111,0.066667,0.044444,0.044444,0.011111,0.011111,0.022222,0.066667,0.044444,0.611111,0.166667,0.322222,0.222222,0.066667,0.088889,0.011111,0.0,0.077778,0.022222,0.088889,0.111111,0.011111,0.0,0.100000,0.033333,0.000000,0.088889,0.044444,0.0,0.0,0.0,0.311111,0.033333,0.055556,4.055556,4.522222,85.688889,20.300000,1.088889,1.188889,2.311111,2.455556,0.655556,0.855556,0.000000,0.002222,0.011111,0.188889,0.055556,0.011111,0.244444,4.522222,4.244444,0.277778,0.044444,0.011111,0.466667,0.133333,0.088889,0.000000,0.000000,0.000000,0.0,3.555556,0.433333,0.533333,2.111111,2.066667,0.077778,0.233333,0.011111,4.055556,0.011111,0.100000,0.011111,0.066667,4.888889,0.366667,2.133333,2.477778,0.622222,0.055556,4.611111,0.055556,0.000000,16.688889,9.844444,0.044444,0.011111,0.033333,0.011111,4.155556,3.977778,0.077778,0,3.0,1,Away
18643,EPL,2020-2021,Arsenal,2020-09-12,G,1.0,0.000000,0.000000,0.000000,0.0,0.366667,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.355556,0.355556,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011

In [21]:
home_stats_attack_df = home_stats_df[home_stats_df['PosNew'].isin(['A', 'M'])]
home_stats_attack_df = home_stats_attack_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
home_stats_defense_df = home_stats_df[home_stats_df['PosNew'].isin(['D', 'M'])]
home_stats_defense_df = home_stats_defense_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
home_stats_attack_df.head()

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog
2447,EPL,2020-2021,Fulham,2020-09-12,0,0.0,0,Home,9.0,0.130472,0.035714,0.011111,0.0,5.218573,2.367038,0.150079,0.011111,0.275533,0.001613,0.001613,0.001111,0.085177,0.000000,3.530462,4.082502,0.109801,3.384537,0.381503,0.137020,0.196518,0.150079,0.089740,0.100851,0.013514,0.035714,0.114343,0.351760,0.237417,2.367038,0.468336,0.558690,1.165021,0.643327,0.275533,0.049228,0.0,0.226306,0.011111,0.161190,0.049462,0.000000,0.0,0.144462,0.186482,0.016129,0.096288,0.089740,0.000000,0.0,0.0,0.528126,0.054480,0.051843,3.530462,4.082502,55.740137,10.037338,1.636407,1.739902,1.600152,1.843063,0.146825,0.341349,0.000000,0.001111,0.057937,0.254011,0.011111,0.011111,0.109801,4.082502,3.935190,0.147312,0.071685,0.000000,0.692649,0.069048,0.096288,0.011111,0.000000,0.011111,0.0,3.417121,0.349710,0.315670,1.246058,2.690694,0.044444,0.000000,0.057937,3.530462,0.000000,0.098214,0.096053,0.027240,5.218573,0.272067,0.895136,2.869005,1.665795,0.032258,5.109613,0.137020,0.000000,17.618140,7.752273,0.060573,0.000000,0.228109,0.174083,4.826612,4.032559,0.232254
2415,EPL,2020-2021,Crystal Palace,2020-09-12,1,1.0,1,Home,8.0,0.044444,0.022222,0.026199,0.0,4.400447,1.181893,0.102588,0.073231,0.201499,0.005556,0.005556,0.011438,0.222822,0.024810,1.874869,3.063897,0.175110,2.730586,0.123611,0.277778,0.444444,0.102588,0.080365,0.013699,0.044444,0.044444,0.000000,0.055556,0.055556,1.181893,0.355665,0.380475,0.564108,0.237310,0.201499,0.000000,0.0,0.201499,0.073231,0.175818,0.105855,0.000000,0.0,0.190197,0.150300,0.044444,0.232244,0.080365,0.000000,0.0,0.0,1.259972,0.190578,0.404086,1.874869,3.063897,42.181303,15.033582,0.814979,1.004168,0.537149,0.987068,0.486821,0.797361,0.011111,0.011438,0.164679,0.176499,0.011111,0.000000,0.175110,3.063897,2.534511,0.529386,0.194744,0.000000,0.663210,0.311031,0.232244,0.311031,0.199919,0.000000,0.0,1.453787,0.203896,1.406214,0.791206,1.893115,0.187610,0.012500,0.000000,1.874869,0.011111,0.155556,0.022222,0.266967,4.400447,0.128078,0.812258,2.197311,1.630775,0.177778,3.984759,0.277778,0.000000,9.800062,5.043167,0.044444,0.033333,0.181944,0.351199,2.890935,2.088728,0.327778
2498,EPL,2020-2021,Liverpool,2020-09-12,2,4.0,1,Home,8.0,0.210706,0.037607,0.011111,0.0,5.360911,2.243772,0.167505,0.052632,0.186028,0.035088,0.018421,0.012222,0.367613,0.041414,3.159498,3.990558,0.524579,3.447795,0.522188,0.234683,0.330004,0.167505,0.088484,0.070069,0.059829,0.037607,0.026496,0.09743

In [22]:
away_stats_attack_df = away_stats_df[away_stats_df['PosNew'].isin(['A', 'M'])]
away_stats_attack_df = away_stats_attack_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
away_stats_defense_df = away_stats_df[away_stats_df['PosNew'].isin(['D', 'M'])]
away_stats_defense_df = away_stats_defense_df.groupby(['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome']).sum().reset_index().sort_values('game_id')
away_stats_attack_df.head()

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog
2334,EPL,2020-2021,Arsenal,2020-09-12,0,3.0,1,Away,8.0,0.145649,0.047183,0.011111,0.0,5.550547,1.741105,0.146756,0.033333,0.101034,0.021882,0.021882,0.015385,0.272828,0.040000,4.053983,4.868448,0.261795,3.754268,0.376154,0.139922,0.228811,0.146756,0.133936,0.026154,0.094961,0.025641,0.025641,0.059491,0.033850,1.741105,0.562071,0.172665,1.193703,0.374737,0.101034,0.000000,0.0,0.101034,0.033333,0.180089,0.013333,0.0,0.0,0.047183,0.132153,0.022739,0.126154,0.133936,0.000000,0.000000,0.0,0.600521,0.011111,0.011111,4.053983,4.868448,71.396035,15.231493,1.719682,1.974647,1.645315,1.869247,0.499064,0.594627,0.026667,0.015385,0.142226,0.400342,0.115043,0.024444,0.261795,4.868448,4.576221,0.292226,0.133932,0.012821,0.781469,0.143850,0.126154,0.146667,0.066667,0.013333,0.000000,3.523548,0.795238,0.549662,1.473166,3.192970,0.047863,0.000000,0.143337,4.053983,0.023932,0.023256,0.011628,0.347782,5.550547,0.074017,1.077183,2.838265,1.816299,0.555921,5.243278,0.151034,0.083333,18.243150,11.613890,0.097265,0.118889,0.102739,0.059328,5.363333,4.335480,0.418501
2580,EPL,2020-2021,Southampton,2020-09-12,1,0.0,0,Away,8.0,0.089683,0.045238,0.000000,0.0,4.828947,1.596032,0.338221,0.011111,0.162364,0.007937,0.007937,0.003413,0.114745,0.000000,2.349749,3.343442,0.190476,2.571429,0.240267,0.104762,0.294444,0.338221,0.113952,0.068713,0.068713,0.200794,0.200000,0.272807,0.072807,1.596032,0.519549,0.347076,0.387135,0.861821,0.162364,0.000000,0.0,0.162364,0.011111,0.349332,0.011111,0.0,0.0,0.082665,0.198079,0.011111,0.093776,0.113952,0.000000,0.000000,0.0,0.843442,0.372222,0.477444,2.349749,3.343442,38.279950,9.114077,1.313325,1.871721,0.724520,0.939724,0.233333,0.384712,0.000000,0.003413,0.034127,0.156349,0.056349,0.011111,0.190476,3.343442,3.243442,0.100000,0.033333,0.000000,0.681078,0.079825,0.093776,0.033333,0.011111,0.022222,0.000000,2.007101,0.530869,0.805472,0.542774,2.214160,0.192063,0.011111,0.022222,2.349749,0.000000,0.022222,0.055556,0.162364,4.828947,0.022222,0.560109,2.564119,1.913116,0.337302,4.706725,0.104762,0.000000,9.621930,4.290769,0.022222,0.011905,0.389683,0.249332,4.029156,3.318254,0.389808
2465,EPL,2020-2021,Leeds United,2020-09-12,2,3.0,0,Away,9.0,0.067509,0.040005,0.000000,0.0,4.364052,2.372125,0.323941,0.023611,0.221470,0.005640,0.005640,0.003333,0.068056,0.045833,2.068930,2.971272,0.117236,1.967029,0.281762,0.168056,0.208060,0.323941,0.123338,0.260325,0.034722,0.028893,0.082

In [23]:
away_stats_defense_df[away_stats_defense_df.isna().any(axis=1)]

,League,Season,Team,Date,game_id,Goal,Win,isHome,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Int,Tkl+Int,Clr,Err,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial_Duels_Won,Aerial_Duels_Lost,Total_Cmp,Total_Att,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Medium_Cmp,Medium_Att,Long_Cmp,Long_Att,Ast,xA,KP,1/3,PPA,CrsPA,Prog,Att,Pass_Types_Live,Pass_Types_Dead,Pass_Types_FK,Pass_Types_TB,Pass_Types_Press,Pass_Types_Sw,Pass_Types_Crs,Pass_Types_CK,Corner_Kicks_In,Corner_Kicks_Out,Corner_Kicks_Str,Height_Ground,Height_Low,Height_High,Body_Parts_Left,Body_Parts_Right,Body_Parts_Head,Body_Parts_TI,Body_Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog


In [24]:
home_stats_attack_df_78 = home_stats_attack_df[home_stats_attack_df['Season'] == '2017-2018']
home_stats_defense_df_78 = home_stats_defense_df[home_stats_defense_df['Season'] == '2017-2018']
away_stats_attack_df_78 = away_stats_attack_df[away_stats_attack_df['Season'] == '2017-2018']
away_stats_defense_df_78 = away_stats_defense_df[away_stats_defense_df['Season'] == '2017-2018']

In [25]:
len(home_stats_attack_df_78), len(home_stats_defense_df_78), len(away_stats_attack_df_78), len(away_stats_defense_df_78)

(1826, 1826, 1826, 1826)

In [26]:
defensive_columns = [el for el in list(home_stats_attack_df_78.columns) if el.startswith('Tackles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Vs_Dribbles')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pressures_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Blocks_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Int')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Tkl+Int')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Clr')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Err')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Aerial_')] +\
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Short_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Medium_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Long_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_Types')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('SCA_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Carries_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Dribbles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Total_')] 

defensive_columns.remove('SCA_GCA')

defensive_columns = list(set(defensive_columns))

In [27]:
offensive_columns = [el for el in list(home_stats_attack_df_78.columns) if el.startswith('SCA_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Passes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Carries_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Dribbles_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Total_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Short_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Medium_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Long_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Pass_Types')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Outcomes_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Touches_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Receiving_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Aerial_')] + \
[el for el in list(home_stats_attack_df_78.columns) if el.startswith('Performance_')]

offensive_columns.remove('SCA_GCA')

In [28]:
# offensive_columns = list(set(offensive_columns + defensive_columns))
# defensive_columns = offensive_columns

In [29]:
len(offensive_columns), len(defensive_columns)

(78, 57)

In [30]:
X_train_home_attack_df_78 = home_stats_attack_df_78[offensive_columns]
X_train_home_defense_df_78 = home_stats_defense_df_78[defensive_columns]
y_train_home_df_78 = home_stats_defense_df_78['Goal']

X_train_away_attack_df_78 = away_stats_attack_df_78[offensive_columns]
X_train_away_defense_df_78 = away_stats_defense_df_78[defensive_columns]
y_train_away_df_78 = away_stats_defense_df_78['Goal']

In [31]:
# X_train_home_attack_df_78_means = X_train_home_attack_df_78.mean()
# X_train_home_defense_df_78_means = X_train_home_defense_df_78.mean()
# X_train_away_attack_df_78_means = X_train_away_attack_df_78.mean()
# X_train_away_defense_df_78_means = X_train_away_defense_df_78.mean()

# X_train_home_attack_df_78_stds = X_train_home_attack_df_78.std()
# X_train_home_defense_df_78_stds = X_train_home_defense_df_78.std()
# X_train_away_attack_df_78_stds = X_train_away_attack_df_78.std()
# X_train_away_defense_df_78_stds = X_train_away_defense_df_78.std()

In [32]:
# X_train_home_attack_df_78 = (X_train_home_attack_df_78 - X_train_home_attack_df_78_means) / X_train_home_attack_df_78_stds
# X_train_home_defense_df_78 = (X_train_home_defense_df_78 - X_train_home_defense_df_78_means) / X_train_home_defense_df_78_stds

# X_train_away_attack_df_78 = (X_train_away_attack_df_78 - X_train_away_attack_df_78_means) / X_train_away_attack_df_78_stds
# X_train_away_defense_df_78 = (X_train_away_defense_df_78 - X_train_away_defense_df_78_means) / X_train_away_defense_df_78_stds


In [33]:
y_train_home_df_78.mean()

1.5301204819277108

In [34]:
y_train_away_df_78.mean()

1.17907995618839

In [35]:
assert not np.any(np.isnan(y_train_away_df_78.values))

In [36]:
import torch
import torch.nn as nn

In [37]:
X_h_a = torch.from_numpy(X_train_home_attack_df_78.values).float()
X_h_d = torch.from_numpy(X_train_home_defense_df_78.values).float()
X_a_a = torch.from_numpy(X_train_away_attack_df_78.values).float()
X_a_d = torch.from_numpy(X_train_away_defense_df_78.values).float()

y_h = torch.from_numpy(y_train_home_df_78.values).float().unsqueeze(1)
y_a = torch.from_numpy(y_train_away_df_78.values).float().unsqueeze(1)
y = torch.cat((y_h, y_a), 1)

In [38]:
# MSE loss
def custom_mse(t1, t2):
    y1 = t2[:, 0]
    y2 = t2[:, 1]
    y1_hat = t1[:, 0]
    y2_hat = t1[:, 1]
    diff1 = y1 - y1_hat
    diff2 = y2 - y2_hat
    return (torch.sum(diff1 * diff1) / diff1.numel()) + (torch.sum(diff2 * diff2) / diff2.numel())

In [39]:
torch.__version__

'1.7.1'

In [40]:
# %%writefile best_model.py

class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        
        self.off_rating = nn.Sequential(
            nn.Linear(78,7),
            nn.BatchNorm1d(7),
            nn.ReLU(),
            nn.Linear(7, 1),
            nn.ReLU()
        )
        self.def_rating = nn.Sequential(
            nn.Linear(57,7),
            nn.BatchNorm1d(7),
            nn.ReLU(),
            nn.Linear(7, 1),
            nn.ReLU()
        )
    
    def forward(self, x_h_a, x_h_d, x_a_a, x_a_d):
        r_h_a = self.off_rating(x_h_a)
#         print(r_h_a.shape)
        r_a_d = self.def_rating(x_a_d)
#         print(r_a_d.shape)
        r_a_a = self.off_rating(x_a_a)
#         print(r_a_a.shape)
        r_h_d = self.def_rating(x_h_d)
#         print(r_h_d.shape)
        
        p1 = 1.5438116100766703 * r_h_a * r_a_d
#         print(p1.shape)
        p2 = 1.17907995618839 * r_a_a * r_h_d
#         print(p2.shape)
#         print(torch.cat((p1, p2), 1).shape)
        
        return torch.cat((p1, p2), 1)

In [44]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

dataset = TensorDataset(X_h_a, X_h_d, X_a_a, X_a_d, y)
train_set, val_set = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset) - int(0.8*len(dataset))])
train_loader = DataLoader(
    train_set,
    batch_size=128
)
val_loader = DataLoader(
    val_set,
    batch_size=128
)

In [47]:
# Repeat for given number of epochs
m = model()
opt = torch.optim.SGD(m.parameters(), lr=1e-4)

In [48]:
best_val_loss = float('inf')
best_m = None

for epoch in range(2000):
    m = m.train()
    train_losses = []

    # Train with batches of data
    for x_h_a, x_h_d, x_a_a, x_a_d, y_e in train_loader:

        # 1. Generate predictions
        pred = m(x_h_a, x_h_d, x_a_a, x_a_d)

        # 2. Calculate loss
        loss = custom_mse(pred, y_e)
        
        train_losses.append(loss.item())

        # 3. Compute gradients
        loss.backward()

        # 4. Update parameters using gradients
        opt.step()

        # 5. Reset the gradients to zero
        opt.zero_grad()

    m = m.eval()
    val_losses = []
    with torch.no_grad():
        for x_h_a, x_h_d, x_a_a, x_a_d, y_e in val_loader:

            # 1. Generate predictions
            pred = m(x_h_a, x_h_d, x_a_a, x_a_d)

            # 2. Calculate loss
            loss = custom_mse(pred, y_e)
            
            val_losses.append(loss.item())
    
    if np.mean(val_losses) < best_val_loss:
        best_val_loss = np.mean(val_losses)
        best_m = copy.deepcopy(m)
        

    # Print the progress
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f} Val Loss: {:.4f}'.format(epoch+1, 1000, np.mean(train_losses), np.mean(val_losses)))

Epoch [100/1000], Loss: 5.8790 Val Loss: 5.1071
Epoch [200/1000], Loss: 3.4073 Val Loss: 3.1027
Epoch [300/1000], Loss: 3.0788 Val Loss: 2.8848
Epoch [400/1000], Loss: 2.9042 Val Loss: 2.7658
Epoch [500/1000], Loss: 2.7817 Val Loss: 2.7184
Epoch [600/1000], Loss: 2.7138 Val Loss: 2.6885
Epoch [700/1000], Loss: 2.6652 Val Loss: 2.6785
Epoch [800/1000], Loss: 2.6270 Val Loss: 2.6272
Epoch [900/1000], Loss: 2.5919 Val Loss: 2.6880
Epoch [1000/1000], Loss: 2.5596 Val Loss: 2.6223
Epoch [1100/1000], Loss: 2.5356 Val Loss: 2.5162
Epoch [1200/1000], Loss: 2.5069 Val Loss: 2.5692
Epoch [1300/1000], Loss: 2.4858 Val Loss: 2.5830
Epoch [1400/1000], Loss: 2.4693 Val Loss: 2.4956
Epoch [1500/1000], Loss: 2.4595 Val Loss: 2.6265
Epoch [1600/1000], Loss: 2.4493 Val Loss: 2.4970
Epoch [1700/1000], Loss: 2.4453 Val Loss: 2.9332
Epoch [1800/1000], Loss: 2.4331 Val Loss: 3.0677
Epoch [1900/1000], Loss: 2.4124 Val Loss: 3.0545
Epoch [2000/1000], Loss: 2.3922 Val Loss: 3.2285


In [49]:
# best_m.train()

m = copy.deepcopy(best_m)
m = m.eval()
torch.save(m.state_dict(), 'best_model6.pth')

In [50]:
m = model()
m.load_state_dict(torch.load('best_model6.pth'))

m = m.eval()

In [51]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [52]:
custom_mse(m(X_h_a, X_h_d, X_a_a, X_a_d).detach(), y)

tensor(2.4586)

In [53]:
home_stats_attack_df_79 = home_stats_attack_df
home_stats_defense_df_79 = home_stats_defense_df
away_stats_attack_df_79 = away_stats_attack_df
away_stats_defense_df_79 = away_stats_defense_df

In [54]:
ratings = pd.concat([home_stats_attack_df_79, away_stats_attack_df_79], ignore_index = True)
ratings['o_rating'] =  m.off_rating(torch.from_numpy(ratings[offensive_columns].values).float()).detach().squeeze(1).numpy()
ratings_o = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'isHome']].copy()

ratings = pd.concat([home_stats_defense_df_79, away_stats_defense_df_79], ignore_index = True)
ratings['d_rating'] =  m.def_rating(torch.from_numpy(ratings[defensive_columns].values).float()).detach().squeeze(1).numpy()
ratings_d = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'd_rating', 'isHome']].copy()

len(ratings_o), len(ratings_d)


(13958, 13958)

In [55]:
ratings = pd.merge(ratings_o, ratings_d, how='inner', on=['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'isHome'])

In [56]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['Season', 'League', 'Team']).mean()
ratings_new['o_rating'].groupby('League', group_keys=False).nlargest(5)

Season     League      Team           
2017-2018  Bundesliga  Bayern Munich      1.516458
                       Leverkusen         1.122914
                       Dortmund           1.104908
                       RB Leipzig         1.085683
                       M'Gladbach         1.075565
           EPL         Manchester City    1.672553
                       Manchester Utd     1.386701
                       Arsenal            1.289217
                       Tottenham          1.246776
                       Liverpool          1.164079
           LaLiga      Real Madrid        1.830290
                       Barcelona          1.580684
                       Villarreal         1.269716
                       Sevilla            1.251124
                       Betis              1.171825
           Ligue1      Paris S-G          1.832029
                       Nice               1.493817
                       Lyon               1.389367
                       Marseille          1

In [57]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['League', 'Season', 'Team']).mean()
ratings_new['d_rating'].groupby('League', group_keys=False).nlargest(5)

League      Season     Team         
Bundesliga  2017-2018  Freiburg         1.030846
                       Hertha BSC       1.020339
                       Wolfsburg        0.997997
                       Schalke 04       0.982254
                       Werder Bremen    0.954994
EPL         2017-2018  Brighton         1.025988
                       Everton          1.023048
                       Swansea City     1.021971
                       Burnley          1.018480
                       Stoke City       1.012656
LaLiga      2017-2018  Real Sociedad    1.051850
                       Leganés          1.047290
                       Girona           1.044670
                       Las Palmas       1.040370
                       Málaga           1.017167
Ligue1      2017-2018  Bordeaux         1.015674
                       Guingamp         0.998815
                       Troyes           0.995796
                       Rennes           0.991930
                       Angers   

In [58]:
# ratings = ratings[['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'isHome']]
# ratings['o_rating'] = - ratings['o_rating']
# ratings['d_rating'] = - ratings['d_rating']
ratings = ratings.sort_values('Date')
ratings

,League,Season,Team,Date,game_id,Goal,Win,o_rating,isHome,d_rating
2467,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,0.633802,Home,0.928669
9446,Ligue1,2017-2018,Toulouse,2017-08-04,2662,2.0,0,0.954708,Away,0.830330
9448,Ligue1,2017-2018,Nice,2017-08-05,2664,0.0,0,1.275465,Away,0.769673
9449,Ligue1,2017-2018,Caen,2017-08-05,2665,0.0,0,0.774932,Away,0.981632
9450,Ligue1,2017-2018,Guingamp,2017-08-05,2666,3.0,1,1.087582,Away,0.812823
...,...,...,...,...,...,...,...,...,...,...
12817,LaLiga,2020-2021,Valladolid,2021-04-28,6122,2.0,0,1.151042,Away,0.869692
12818,LaLiga,2020-2021,Granada,2021-04-29,6123,2.0,1,0.936062,Away,0.968451
5839,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.451126,Home,0.642033
328,EPL,2020-2021,Southampton,2021-04-30,330,1.0,0,0.908411,Home,1.136841


In [59]:
ratings['AvgORating'] = ratings.groupby(['Team'])['o_rating'].transform(lambda x: x.rolling(30, 1).mean().shift(1, fill_value=0))
ratings['AvgDRating'] = ratings.groupby(['Team'])['d_rating'].transform(lambda x: x.rolling(30, 1).mean().shift(1, fill_value=0))


In [60]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['Season', 'League', 'Team']).mean()
ratings_new['o_rating'].groupby('League', group_keys=False).nlargest(5)

Season     League      Team           
2017-2018  Bundesliga  Bayern Munich      1.516458
                       Leverkusen         1.122914
                       Dortmund           1.104908
                       RB Leipzig         1.085683
                       M'Gladbach         1.075565
           EPL         Manchester City    1.672553
                       Manchester Utd     1.386701
                       Arsenal            1.289217
                       Tottenham          1.246776
                       Liverpool          1.164079
           LaLiga      Real Madrid        1.830290
                       Barcelona          1.580684
                       Villarreal         1.269716
                       Sevilla            1.251124
                       Betis              1.171825
           Ligue1      Paris S-G          1.832029
                       Nice               1.493817
                       Lyon               1.389367
                       Marseille          1

In [64]:
ratings_new = ratings[ratings['Season'] == '2017-2018'].groupby(['League', 'Season', 'Team']).mean()
ratings_new['d_rating'].groupby('League', group_keys=False).nsmallest(5)

League      Season     Team           
Bundesliga  2017-2018  Bayern Munich      0.815757
                       Hannover 96        0.895439
                       Hoffenheim         0.903160
                       RB Leipzig         0.909372
                       Augsburg           0.917462
EPL         2017-2018  Manchester City    0.794200
                       Liverpool          0.852791
                       Tottenham          0.875947
                       Chelsea            0.890470
                       Arsenal            0.908641
LaLiga      2017-2018  Real Madrid        0.861930
                       Barcelona          0.901173
                       Celta Vigo         0.943000
                       Sevilla            0.951586
                       Atlético Madrid    0.959558
Ligue1      2017-2018  Paris S-G          0.857564
                       Nice               0.921912
                       Metz               0.923244
                       Montpellier        0

In [65]:
ratings_home = ratings[ratings['isHome'] == 'Home'][['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'AvgORating', 'AvgDRating']]
ratings_away = ratings[ratings['isHome'] == 'Away'][['League', 'Season', 'Team', 'Date', 'game_id', 'Goal', 'Win', 'o_rating', 'd_rating', 'AvgORating', 'AvgDRating']]


df89 = pd.merge(ratings_home, ratings_away, on = ['League', 'Season', 'Date', 'game_id'], how='inner', suffixes=('_Home', '_Away'),)
final_data = df89.copy()

df89 = df89[df89['Season'] == '2018-2019']


In [66]:
final_data

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,AvgORating_Home,AvgDRating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away,AvgORating_Away,AvgDRating_Away
0,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,0.633802,0.928669,0.000000,0.000000,Toulouse,2.0,0,0.954708,0.830330,0.000000,0.000000
1,Ligue1,2017-2018,Paris S-G,2017-08-05,2663,2.0,1,1.862248,0.790499,0.000000,0.000000,Amiens,0.0,0,0.798460,0.968143,0.000000,0.000000
2,Ligue1,2017-2018,Saint-Étienne,2017-08-05,2664,1.0,1,1.351762,0.977121,0.000000,0.000000,Nice,0.0,0,1.275465,0.769673,0.000000,0.000000
3,Ligue1,2017-2018,Montpellier,2017-08-05,2665,1.0,1,0.701402,0.994781,0.000000,0.000000,Caen,0.0,0,0.774932,0.981632,0.000000,0.000000
4,Ligue1,2017-2018,Metz,2017-08-05,2666,1.0,0,0.722225,0.938748,0.000000,0.000000,Guingamp,3.0,1,1.087582,0.812823,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,LaLiga,2020-2021,Eibar,2021-04-26,6121,0.0,0,0.834816,1.138296,0.887042,0.938492,Real Sociedad,1.0,1,0.810999,1.169136,0.948072,0.948217
6975,EPL,2020-2021,Leicester City,2021-04-26,329,2.0,1,0.909778,0.943653,1.055599,0.921017,Crystal Palace,1.0,0,0.873620,0.826551,1.024181,0.968191
6976,LaLiga,2020-2021,Athletic Club,2021-04-28,6122,2.0,0,0.931615,1.008721,0.951837,0.945548,Valladolid,2.0,0,1.151042,0.869692,0.936250,0.949630
6977,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.451126,0.642033,1.906406,0.776333,Granada,2.0,1,0.936062,0.968451,0.871670,0.949231


In [67]:
final_data['Season'].value_counts()

2017-2018    1826
2018-2019    1826
2019-2020    1723
2020-2021    1604
Name: Season, dtype: int64

In [68]:
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [69]:
# Confirming with the results shown in class
cols_for_pred = [
    'I(1.5* (AvgORating_Home * AvgDRating_Away) - 1.1*(AvgORating_Away * AvgDRating_Home))', 
#     'I(AvgORating_Home - AvgORating_Away)',
#     'I(AvgDRating_Home - AvgDRating_Away)',
#     'I(o_rating_Home - o_rating_Away)',
#     'I(d_rating_Home - d_rating_Away)'
]

glm_str = "Win_Home ~ " + '+'.join(cols_for_pred)

result_glm = smf.glm(glm_str, 
             data = df89, 
             family = sm.families.Binomial()
).fit()
result_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Win_Home   No. Observations:                 1826
Model:                            GLM   Df Residuals:                     1824
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1179.4
Date:                Tue, 04 May 2021   Deviance:                       2358.7
Time:                        21:23:41   Pearson chi2:                 1.83e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
============================================================================================================================================================
                                                                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                   -0.7828      0.070    -11.106      0.000      -0.921      -0.645
I(1.5 * (AvgORating_Home * AvgDRating_Away) - 1.1 * (AvgORating_Away * AvgDRating_Home))     1.3612      0.120     11.347      0.000       1.126       1.596
============================================================================================================================================================
"""

In [70]:
print("Test Brier Score: ", brier_score_loss(df89['Win_Home'], result_glm.predict(df89)))


Test Brier Score:  0.22725816381356773


In [71]:
result_glm.predict(df89)

1826    0.623633
1827    0.682349
1828    0.308911
1829    0.491592
1830    0.264494
          ...   
3647    0.623831
3648    0.348135
3649    0.444078
3650    0.482366
3651    0.468742
Length: 1826, dtype: float64

In [72]:
df89['Win_Home']

1826    1
1827    1
1828    0
1829    1
1830    0
       ..
3647    1
3648    0
3649    0
3650    1
3651    1
Name: Win_Home, Length: 1826, dtype: int64

In [73]:
print("Testing Acc: ", accuracy_score(df89['Win_Home'], 1 * (result_glm.predict(df89) > 0.5)))
s

Testing Acc:  0.6308871851040526


In [74]:
# Confirming with the results shown in class
cols_for_pred = [
    'I(AvgORating_Home - AvgORating_Away)', 
    'I(AvgDRating_Home - AvgDRating_Away)'
]

glm_str = "Win_Home ~ " + '+'.join(cols_for_pred)

result_glm = smf.glm(glm_str, 
             data = df89, 
             family = sm.families.Binomial()
).fit()
result_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Win_Home   No. Observations:                 1826
Model:                            GLM   Df Residuals:                     1823
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1182.9
Date:                Tue, 04 May 2021   Deviance:                       2365.9
Time:                        21:23:49   Pearson chi2:                 1.83e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -0.2248      0.049     -4.587      0.000      -0.321      -0.129
I(AvgORating_Home - AvgORating_Away)     1.6742      0.171      9.806      0.000       1.340       2.009
I(AvgDRating_Home - AvgDRating_Away)    -1.8302      0.474     -3.862      0.000      -2.759      -0.901
========================================================================================================
"""

In [75]:
print("Test Brier Score: ", brier_score_loss(df89['Win_Home'], result_glm.predict(df89)))


Test Brier Score:  0.22816636420885544


In [85]:
df89['Win_Home'].value_counts() 

0    1009
1     817
Name: Win_Home, dtype: int64

In [76]:
print("Testing Acc: ", accuracy_score(df89['Win_Home'], 1 * (result_glm.predict(df89) > 0.5)))


Testing Acc:  0.624315443592552


In [84]:
(1 * (result_glm.predict(df89) > 0.5)).value_counts()

0    1313
1     513
dtype: int64

In [77]:
final_data.drop(['AvgORating_Home', 'AvgORating_Away', 'AvgDRating_Home', 'AvgDRating_Away'], 1).to_csv('offensive_defensive_ratings6.csv', index=False)


In [78]:
final_data

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,AvgORating_Home,AvgDRating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away,AvgORating_Away,AvgDRating_Away
0,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,0.633802,0.928669,0.000000,0.000000,Toulouse,2.0,0,0.954708,0.830330,0.000000,0.000000
1,Ligue1,2017-2018,Paris S-G,2017-08-05,2663,2.0,1,1.862248,0.790499,0.000000,0.000000,Amiens,0.0,0,0.798460,0.968143,0.000000,0.000000
2,Ligue1,2017-2018,Saint-Étienne,2017-08-05,2664,1.0,1,1.351762,0.977121,0.000000,0.000000,Nice,0.0,0,1.275465,0.769673,0.000000,0.000000
3,Ligue1,2017-2018,Montpellier,2017-08-05,2665,1.0,1,0.701402,0.994781,0.000000,0.000000,Caen,0.0,0,0.774932,0.981632,0.000000,0.000000
4,Ligue1,2017-2018,Metz,2017-08-05,2666,1.0,0,0.722225,0.938748,0.000000,0.000000,Guingamp,3.0,1,1.087582,0.812823,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,LaLiga,2020-2021,Eibar,2021-04-26,6121,0.0,0,0.834816,1.138296,0.887042,0.938492,Real Sociedad,1.0,1,0.810999,1.169136,0.948072,0.948217
6975,EPL,2020-2021,Leicester City,2021-04-26,329,2.0,1,0.909778,0.943653,1.055599,0.921017,Crystal Palace,1.0,0,0.873620,0.826551,1.024181,0.968191
6976,LaLiga,2020-2021,Athletic Club,2021-04-28,6122,2.0,0,0.931615,1.008721,0.951837,0.945548,Valladolid,2.0,0,1.151042,0.869692,0.936250,0.949630
6977,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.451126,0.642033,1.906406,0.776333,Granada,2.0,1,0.936062,0.968451,0.871670,0.949231


In [72]:
final_data['League'].unique()

array(['Ligue1', 'EPL', 'Bundesliga', 'LaLiga', 'SerieA'], dtype=object)

In [79]:
final_data.sort_values('o_rating_Home', ascending=False)

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,AvgORating_Home,AvgDRating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away,AvgORating_Away,AvgDRating_Away
3651,SerieA,2018-2019,Atalanta,2019-05-26,5413,3.0,1,5.266490,0.434893,1.143284,0.917525,Sassuolo,1.0,0,0.561146,1.149939,1.179208,0.975696
2300,SerieA,2018-2019,Milan,2018-10-28,5130,3.0,1,5.114106,0.858412,1.357432,0.944250,Sampdoria,2.0,0,1.333817,0.930210,1.163662,0.941104
5511,SerieA,2020-2021,Bologna,2020-09-28,4290,4.0,1,4.772768,0.868749,1.060925,0.932351,Parma,1.0,0,0.672312,0.952258,1.029844,0.906663
2493,Bundesliga,2018-2019,RB Leipzig,2018-12-02,3772,2.0,1,4.730498,0.934838,1.154419,0.922450,M'Gladbach,0.0,0,1.097606,0.994073,0.972063,0.926728
5071,LaLiga,2019-2020,Athletic Club,2020-06-20,6468,1.0,1,4.586791,0.829915,0.833335,0.995999,Betis,0.0,0,0.787043,0.869780,1.087306,1.010835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6882,LaLiga,2020-2021,Getafe,2021-04-18,6101,0.0,0,0.070307,1.068511,0.884608,0.952846,Real Madrid,0.0,0,0.948462,1.030089,1.471064,0.870453
6510,Bundesliga,2020-2021,Eint Frankfurt,2021-02-20,3236,2.0,1,0.000000,1.540571,0.960348,0.884855,Bayern Munich,1.0,0,1.894409,0.831471,1.163574,0.816504
1372,Ligue1,2017-2018,Toulouse,2018-03-11,2951,1.0,0,0.000000,1.180296,0.895296,0.955922,Marseille,2.0,1,1.140531,0.905926,1.350385,0.949165
908,EPL,2017-2018,Crystal Palace,2017-12-31,1347,0.0,0,0.000000,1.656141,0.869162,1.004194,Manchester City,0.0,0,0.825803,0.765191,1.739367,0.796656


In [86]:
pd.read_csv('offensive_defensive_ratings6.csv', parse_dates=['Date'])

,League,Season,Team_Home,Date,game_id,Goal_Home,Win_Home,o_rating_Home,d_rating_Home,Team_Away,Goal_Away,Win_Away,o_rating_Away,d_rating_Away
0,Ligue1,2017-2018,Monaco,2017-08-04,2662,3.0,1,0.633802,0.928669,Toulouse,2.0,0,0.954708,0.830330
1,Ligue1,2017-2018,Paris S-G,2017-08-05,2663,2.0,1,1.862248,0.790499,Amiens,0.0,0,0.798460,0.968143
2,Ligue1,2017-2018,Saint-Étienne,2017-08-05,2664,1.0,1,1.351762,0.977121,Nice,0.0,0,1.275465,0.769673
3,Ligue1,2017-2018,Montpellier,2017-08-05,2665,1.0,1,0.701402,0.994781,Caen,0.0,0,0.774932,0.981632
4,Ligue1,2017-2018,Metz,2017-08-05,2666,1.0,0,0.722225,0.938748,Guingamp,3.0,1,1.087582,0.812823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,LaLiga,2020-2021,Eibar,2021-04-26,6121,0.0,0,0.834816,1.138296,Real Sociedad,1.0,1,0.810999,1.169136
6975,EPL,2020-2021,Leicester City,2021-04-26,329,2.0,1,0.909778,0.943653,Crystal Palace,1.0,0,0.873620,0.826551
6976,LaLiga,2020-2021,Athletic Club,2021-04-28,6122,2.0,0,0.931615,1.008721,Valladolid,2.0,0,1.151042,0.869692
6977,LaLiga,2020-2021,Barcelona,2021-04-29,6123,1.0,0,1.451126,0.642033,Granada,2.0,1,0.936062,0.968451
